<a href="https://colab.research.google.com/github/JasperAD11/Sentiment-Across-Signals-Neural-Networks-vs.-LLMs/blob/main/Part2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from model import create_ensemble_model, predict_ensemble_model

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import TextVectorization, Input, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.initializers import Constant
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
df=pd.read_csv("df_merged.csv")

In [5]:
joint_model=create_ensemble_model("best_model_binary.h5","best_model_multi.h5")

In [3]:
df

,filename,sentiment,emotion,text,whisper_transcription
0,audio1.m4a,positive,approval,That was a fantastic presentation. I'm really ...,That was a fantastic presentation. I'm really...
1,audio2.m4a,positive,annoyance,I cannot believe this is happening again. It i...,I cannot believe this is happening again. It ...
2,audio3.m4a,positive,disproval,I really don't think this is the right decision.,I really don't think this is the right decision.
3,audio4.m4a,positive,gratitude,Thank you so much for your help. I truly appre...,Thank you so much for your help. I truly appr...
4,audio5.m4a,positive,curiosity,I wonder how that works. I'd love to know more...,I wonder how that works. I'd love to know mor...
...,...,...,...,...,...
117,audio119.m4a,negative,annoyance,"It feels like I'm stuck in a loop, unable to e...","It feels like I'm stuck in a loop, unable to ..."
118,audio120.m4a,negative,annoyance,The more I try the worse it gets.,The more I try the worse it gets.
119,audio121.m4a,negative,annoyance,It's impossible to keep up with everything.,It's impossible to keep up with everything.
120,audio122.m4a,negative,annoyance,I'm getting tired of repeating myself.,I'm getting tired of repeating myself.


In [19]:
max_vocab = 20000
sequence_len = 300

vectorizer = TextVectorization(
    max_tokens=max_vocab,
    output_mode='int',
    output_sequence_length=sequence_len
)

vectorizer.adapt(df["whisper_transcription"])

df["sentiment_transcript"] = df["whisper_transcription"].apply(lambda x: predict_ensemble_model(joint_model, [x], vectorizer)['sentiment'])
df['sentiment_match_transcript'] = (df['sentiment_transcript'] == df['sentiment']).astype(int)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━

In [20]:
df["sentiment_original"] = df["text"].apply(lambda x: predict_ensemble_model(joint_model, [x], vectorizer)['sentiment'])
df['sentiment_match_original'] = (df['sentiment_original'] == df['sentiment']).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━

In [23]:
df["emotion_transcript"] = df["whisper_transcription"].apply(lambda x: predict_ensemble_model(joint_model, [x], vectorizer)['emotion'])
df['emotion_match_transcript'] = df.apply(lambda row: 1 if row['emotion'] in row['emotion_transcript'] else 0, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━

In [27]:
df["emotion_original"] = df["text"].apply(lambda x: predict_ensemble_model(joint_model, [x], vectorizer)['emotion'])
df['emotion_match_original'] = df.apply(lambda row: 1 if row['emotion'] in row['emotion_original'] else 0, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━

In [28]:
df

,filename,sentiment,emotion,text,whisper_transcription,sentiment_transcript,sentiment_original,sentiment_match_transcript,sentiment_match_original,emotion_transcript,emotion_match_transcript,emotion_original,emotion_match_original
0,audio1.m4a,positive,approval,That was a fantastic presentation. I'm really ...,That was a fantastic presentation. I'm really...,positive,positive,1,1,[neutral],0,[neutral],0
1,audio2.m4a,positive,annoyance,I cannot believe this is happening again. It i...,I cannot believe this is happening again. It ...,positive,positive,1,1,[admiration],0,[admiration],0
2,audio3.m4a,positive,disproval,I really don't think this is the right decision.,I really don't think this is the right decision.,positive,positive,1,1,[neutral],0,[neutral],0
3,audio4.m4a,positive,gratitude,Thank you so much for your help. I truly appre...,Thank you so much for your help. I truly appr...,positive,positive,1,1,[neutral],0,[neutral],0
4,audio5.m4a,positive,curiosity,I wonder how that works. I'd love to know more...,I wonder how that works. I'd love to know mor...,positive,positive,1,1,[approval],0,[approval],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,audio119.m4a,negative,annoyance,"It feels like I'm stuck in a loop, unable to e...","It feels like I'm stuck in a loop, unable to ...",positive,positive,0,0,[admiration],0,[admiration],0
118,audio120.m4a,negative,annoyance,The more I try the worse it gets.,The more I try the worse it gets.,positive,positive,0,0,[neutral],0,[neutral],0
119,audio121.m4a,negative,annoyance,It's impossible to keep up with everything.,It's impossible to keep up with everything.,positive,positive,0,0,[neutral],0,[neutral],0
120,audio122.m4a,negative,annoyance,I'm getting tired of repeating myself.,I'm getting tired of repeating myself.,positive,positive,0,0,[neutral],0,[neutral],0


In [29]:
sent_original_perc = df['sentiment_match_original'].mean()
sent_trans_perc = df['sentiment_match_transcript'].mean()
emo_original_perc = df['emotion_match_original'].mean()
emo_trans_perc = df['emotion_match_transcript'].mean()

In [30]:
print(sent_original_perc)
print(sent_trans_perc)
print(emo_original_perc)
print(emo_trans_perc)

0.5327868852459017
0.5409836065573771
0.01639344262295082
0.01639344262295082
